<!--

### Requirements:

You need to .....

1. simualtion track
2. hardware track
    - DB21M up and running
    - 2 AprilTags

### Intended outcomes:

Calibrate the Duckiebot kinematic model using the data from the wheel encoders.

### Steps:

1. Theory  
2. Approach
3. Implementation
4. Validation



1. Theory -> Review odometry class
2. Approach -> 
    - Use wheel encoders to estimate the pose of the DB using deadrecknonig approximation. 
    - Use estimate and ground truth to best fit

3. Implementation ->
    - Read the data from the wheel encoders 
        - Sim:
        - HW :
            
    - Obtain the gorund truth from AprilTags


4. Validation

1. Theoretical understanding of the Duckiebot kinematic model, in particular what the parameters _R_ and _L_ represent.
2. Approach:
    - Wheel encoders and deadrecknonig
    
    - Straight path 
    - Curved path (e.g., sinusoidal)

3. Validation of the resulting parameters.

-->

<p style="text-align: right">
  <img src="../images/dtlogo.png" alt="Logo" width="200">
</p>

# 🚙 💻 05 - Wheel encoder based odometry

"Odometry" is the problem of "measuring the path", or evolution of the pose in time, of the robot. 

We can solve the odometry problem by using the measurements from wheel encoders. We use the dead-reckoning model to estimate the evolution of the pose in time through an iterative procedure, such that:

<p style="text-align:center;"><img src="../images/odometry/odometry-1.png" width="500" alt="odometry"></p>   

$$ x_{k+1} = x_k + \Delta x_k $$
$$ y_{k+1} = y_k + \Delta y_k $$
$$ \theta_{k+1} = \theta_k + \Delta \theta_k $$

Where initial conditions ($x_0$, $y_0$, $\theta_0$) are assumed to be known. The increments can be calculated by:

1. Determining the rotation of each wheel through the wheel encoder mesurements

$$\Delta \phi_k = N_k \cdot \alpha$$

where $N_k$ is the number of pulses, or "ticks", measured from the encoders in the $k-th$ time interval, $\alpha = \frac{2 \pi}{N_{tot}}$ is the rotation per tick, and $N_{tot}$ the total number of ticks per revolution ($N_{tot} = 135$ for the wheel encoders we will be using). This relation is evaluated for each wheel, yielding $\Delta \phi_{l,k}$ and $\Delta \phi_{r,k}$ for the left and right wheels respectively.

2. Deriving the total distance travelled by each wheel

<p style="text-align:center;"><img src="../images/odometry/odometry-d.png" width="300" alt="odometry"></p> 

Assuming the wheel radii are the same (equal to $R$) for both wheels, the distance travelled by each wheel is given by:

$$ d_{l/r, k} = R \cdot \Delta \phi_{l/r,k}$$

3. Find the rotation and distance travelled by the robot (frame)

<p style="text-align:center;"><img src="../images/odometry/odometry-2.png" width="300" alt="odometry"></p>    

Under the assumption of no slipping of the robot wheel, we can derive the distance travelled by the origin of the robot frame (point $A$) and the rotation of the robot $\Delta \theta$:

$$ d_{A, k} = \frac{d_{r,k} + d_{l,k}}{2} $$
$$ \Delta \theta_{k} = \frac{d_{r,k} - d_{l,k}}{2L}$$

4. Express the robot motion in the world reference frame

<p style="text-align:center;"><img src="../images/odometry/odometry-3.png" width="300" alt="odometry"></p>

Finally, we can express the estimated motion in the world reference frame and find:

$$ \Delta x_k = d_{A, k} \cos\theta_k $$
$$ \Delta y_k = d_{A, k} \sin\theta_k $$

# Let's get started!

In this activity you will write a function that produces an estimate of the pose of the Duckiebot, given mesurements from the wheel encoders and an initial position:

In [ ]:
x0 = y0 = 0 # meters
theta0 = 0 # radians

## 1. Determining the rotation of each wheel through the wheel encoder mesurements

We've seen how to read wheel encoder data. Let's now use it to measure how much each wheel has rotated. 

### Wheel encoder calibration factor

Remember that there are 135 ticks per revolution on the wheel encoders we are using. 

In [4]:
import numpy as np #TODO remove solution before publishing
 
N_tot = 135 # total number of ticks per revolution
alpha = 2 * np.pi / N_tot # wheel rotation per tick in radians

print(f"The angular resolution of our encoders is: {np.rad2deg(alpha)} degrees")

The angular resolution of our encoders is: 2.6666666666666665 degrees


Solution: The angular resolution of our encoders is: 2.6666666666666665 degrees

Let's suppose that at the current update the encoders have produced the following measurements:

In [8]:
ticks_left = 2
prev_tick_left = 0

ticks_right = 7
prev_tick_right = 4

How much did each wheel rotate? (Express your answer in degrees - machines always use radians, humans make sense of degrees better. Mixing these up is a very very common source of error!).

In [9]:
# write function to get delta ticks

delta_ticks_left = ticks_left-prev_tick_left # delta ticks of left wheel in arbitrary dt
delta_ticks_right = ticks_right-prev_tick_right # delta ticks of right wheel in same dt

rotation_wheel_left = alpha * delta_ticks_left # total rotation of left wheel in dt
rotation_wheel_right = alpha * delta_ticks_right # total rotation of right wheel in dt

print(f"The left wheel rotated: {np.rad2deg(rotation_wheel_left)} degrees")
print(f"The right wheel rotated: {np.rad2deg(rotation_wheel_right)} degrees")

The left wheel rotated: 5.333333333333333 degrees
The right wheel rotated: 8.0 degrees


Solution: 

The left wheel rotated: 5.333333333333333 degrees

The right wheel rotated: 8.0 degrees

## 2. Evaluate distance travelled by each wheel

Now let's calculate the distance travelled by each wheel. It depends on the wheel radii. We need to determine it! We could use advanced odometry calibration procedures, but let's take it a step at the time. 

Take a ruler and measure you wheel radii (let's assume they are the same):

In [10]:
R = 0.033 # insert value measured by ruler, in *meters*

Note: the default value used in simulation and on the robot is $R = 0.0318 \text{m}$.

In [13]:
# What is the distance travelled by each wheel?

d_left = R * rotation_wheel_left 
d_right = R * rotation_wheel_right

print(f"The left wheel travelled: {d_left} meters")
print(f"The right wheel rotated: {d_right} meters")

The left wheel travelled: 0.00307177948351002 meters
The right wheel rotated: 0.004607669225265031 meters


### 🚙 Save your new value of `R`

If you have a Duckiebot, let's make sure it remembers its new wheel radius! You should already know how to do this from [wheel calibration tutorial](../03-Wheel-Calibration/wheels_calibration.ipynb). 

Else, you can follow this slightly more straightforward approach. Power you Duckiebot on, make sure it is connected to the network and you can ping it, then open a terminal **on your computer** and type:

    dts start_gui_tools ROBOTNAME
    
    rosparam set /ROBOTNAME/kinematics_node/radius R-value
    
where `R-value` is the value of the wheel radius you measured (expressed in meters). You can then save it with: 

    rosservice call /ROBOTNAME/kinematics_node/save_calibration
    
and finally verify that it has been saved by opening the `ROBOTNAME.yaml` file in your Dashboard > File Manager > Calibrations > Kinematics page.

You can keep the terminal you just used open, so we can save the baseline measurement too. Let's keep going!

## 3. Find the rotation and distance travelled by the robot (frame)

If you have previoulsy set your robot's gain so that the wheels do not slip, the travelled distance of point $A$ (origin of the robot frame) will be given by the average of the distances travelled by the wheels:

In [15]:
# How much has the robot travelled? 

d_A = (d_left + d_right)/2

print(f"The robot has travelled: {d_A} meters")

The robot has travelled: 0.0038397243543875255 meters


To calculate the rotation of the robot we need to measure the baseline - or the distance from the ceter of the two wheels. 




To do so we need to measure the baseline that is the distance between the center of the two wheels.
Take a ruler and mesure it! 

In [1]:
baseline_wheel2wheel = 0.108 #  Distance between the center of the two wheels, expressed in meters (e.g., 10.8cm)

**JT TODO: add instructions on how to go to VNC and set and save the new L** (or directly through start_gui_tools)

In [ ]:
Delta_Theta = (d_right-d_left)/baseline_wheel2wheel

### Step 10

Write in the cell below the code to estimate the pose of the Duckiebot using data given by the wheel encoders.

**DO NOT CHANGE THE NAME OF THE FUNCTION**

In [1]:
def DeltaPhi(encoder_msg, prev_ticks):
    """
        Args:
            encoder_msg (ROS encoder message)
            prev_ticks (Current ticks)
        Return:
            rotation_wheel (double) Rotation of the wheel
            ticks (int) current number of ticks
    """
    ticks = encoder_msg.data

    delta_ticks = ticks-prev_ticks

    N_tot = encoder_msg.resolution

    alpha = 2*np.pi/N_tot

    rotation_wheel = alpha*delta_ticks
    
    return rotation_wheel, ticks

In [1]:
# The function written in this cell will actually be ran on your robot (sim or real). 
# Put together the steps above and write your odometry function! 

import numpy as np 

# DO NOT CHANGE THE NAME OF THIS FUNCTION OR THINGS WILL BREAK

def poseEstimation( R, # radius of wheel (assumed identical)
                    baseline_wheel2wheel, # distance from wheel to wheel (center); 2L of the theory
                    x_prev, # previous estimate - assume given
                    y_prev, # previous estimate - assume given
                    theta_prev, # previous estimate - assume given
                    delta_phi_left, # previous estimate - assume given
                    delta_phi_right):
    """
        Calculate the current Duckiebot pose using dead reckoning approach.

        Returns x,y,theta current estimates:
            x_curr, y_curr, theta_curr (:double: values)
    """
    x_curr = x_prev + R*(delta_phi_left+delta_phi_right)*np.cos(theta_prev)/2
    y_curr = y_prev + R*(delta_phi_left+delta_phi_right)*np.sin(theta_prev)/2
    theta_curr = theta_prev + R*(delta_phi_right-delta_phi_left)/baseline_wheel2wheel
    
    return x_curr, y_curr, theta_curr

### Test the poseEstimation function

Unit tests are useful to check if the piece of code you write does its intended job.
If the tests are good you will not see any output.

Let's see if the function you wrote above passes the following test!




In [17]:
from unit_test import UnitTestOdometry

R = 0.033
baseline_wheel2wheel = 0.108

UnitTestOdometry(R, baseline_wheel2wheel, poseEstimation)


ModuleNotFoundError: No module named 'unit_test'

### Run the Activity

** TODO: Jacopo write the testing experience both in simulation and on the real robot**

Objective is drive robot around and see the odometry track

In [ ]:
! cd .. && dts exercises build

### Run the Activity in simulation

In [ ]:
! cd .. && dts exercises test --sim

After running the above command, open the following link http://localhost:8087 you will visualize the VNC. Then from the VNC open RVIZ, press the `Add` on the bottom left side of the window, after that go to the tab `topics` and select `/DUCKIEBOT_NAME/encoder_localization`. Finally open the joystick (on the Desktop) and move the robot. You will see that the pose arrow start moving according to the given commands.

### Run the activity on the Duckiebot

In [ ]:
! cd .. && dts exercises test --duckiebot_name ![DB_NAME] 

After running the above command, open the following link http://localhost:8087 you will visualize the VNC. Then from the VNC open RVIZ, press the `Add` on the bottom left side of the window, after that go to the tab `topics` and select `/DUCKIEBOT_NAME/encoder_localization`. Finally open the joystick (on the Desktop) and move the robot. You will see that the pose arrow start moving according to the given commands.